In [1]:
RAW_DATA = r'M:\Backup ISI402\12X\Qual\Life test\12X02\dRdP_LRT2_260mW-2.HGA Lifecycle.dRdP-low.csv'
out_path = r'M:\Backup ISI402\12X\Qual\Life test\12X02\dRdP_LRT2_260mW-2.HGA Lifecycle.dRdP-low.xlsx'

In [2]:
import xlsxwriter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import openpyxl
from openpyxl.chart import BarChart3D,Reference

In [3]:
df = pd.read_csv(RAW_DATA, header=[2])
#df = pd.read_csv(RAW_DATA, header=[2], sep=r"[| ^]", engine='python')

#df = df[['Hd','Cycles','Part ID','Test Time','Test','Date','Time']]
df = df.iloc[:,[0,1,4,5,6,7,16,32,44]]
#cols = {0:'Time Stamp(S)',30:'Resistance (Ohms)'}
#df.rename(columns=cols, inplace=True)
df.columns = ['Hd','Part ID','Test Time','Test','Date','Time','TimeStamp (s)','Bias (mA)','Resistance (Ohm)']
df = df.sort_values(['Hd','TimeStamp (s)'])
df['TTF'] = df['TimeStamp (s)']/3600
grouped_df = df.groupby(['Part ID']).first()
df = pd.merge(df, grouped_df, on=['Part ID'] , suffixes=('','_REF'))
#df['Resistance (Ohm)'] = df['Resistance (Ohm)'].replace(np.Infinity, 1000)
df['Delta_Res(%)'] = (df['Resistance (Ohm)']/df['Resistance (Ohm)_REF'] - 1)*100
df['P/F Status'] = np.where(abs(df['Delta_Res(%)'])>=20,1,0)
df['P/F Status'] = df.groupby(['Hd'])['P/F Status'].cumsum()
df['P/F Status'] = np.where(df['Hd']!=df['Hd'].shift(),0,np.where(df['Hd']!=df['Hd'].shift(-1),1,np.where(df['P/F Status']>0,1,0)))
df['P/F_Flag'] = np.where(df['Hd']!=df['Hd'].shift(),0,np.where(df['P/F Status']!=df['P/F Status'].shift(),1,0))



df['F/S'] = np.where((df['P/F_Flag']==1)&(abs(df['Delta_Res(%)'])>=20),'F','S')

#df['Stress(mV)'] = df['Test'].str[14:17]
df['Power (mW)'] = (df['Bias (mA)']/1000)**2*df['Resistance (Ohm)']*1000
df['Selected'] = df['P/F_Flag'].shift(-1)
df = df[['Hd','Part ID','Test Time','Test','Date','Time','TimeStamp (s)','Resistance (Ohm)','TTF','Resistance (Ohm)_REF','Delta_Res(%)','P/F Status','P/F_Flag','F/S','Power (mW)','Selected']]

dy = df.loc[df['P/F_Flag'] == 1]
table = pd.pivot_table(dy, values='Part ID', index=['F/S','TTF'], columns=[], aggfunc='count', margins=False)

In [4]:
writer = pd.ExcelWriter(out_path, engine='xlsxwriter')
df.to_excel(writer, sheet_name='ALL', index=None)
table.to_excel(writer, sheet_name='SUM')
dy.to_excel(writer, sheet_name='TTF')
writer.save()